In [ ]:
#I obtained the frequency distributions of different parts of speech from this cell. This is optional to read.

from sklearn.preprocessing import scale
from sklearn.naive_bayes import MultinomialNB
from collections import Counter
from nltk import FreqDist
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from string import punctuation
import csv
import re
import nltk
from sklearn.svm import LinearSVC
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn import tree
from sklearn import metrics
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score,f1_score

# "id","comment_text","toxic","severe_toxic","obscene","threat","insult","identity_hate"

# read dataset from file
scoredComments = []
scores= []


scoredCommentsTokenized = []
stop_words = set(stopwords.words('english'))

lemma = WordNetLemmatizer()

dataset = pd.read_csv("train.csv")


#display(dataset)
#print(dataset.describe())


toxicSum = dataset["toxic"].sum()
sevToxicSum = dataset["severe_toxic"].sum()
obsceneSum = dataset["obscene"].sum()
threatSum = dataset["threat"].sum()
insSum = dataset["insult"].sum()
idhateSum = dataset["identity_hate"].sum()

total = 159570
print("Toxic rate:" + str(toxicSum / 159570))
print("Severe Toxic rate:" + str(sevToxicSum / 159570))
print("Obscene rate:" + str(obsceneSum/ 159570))
print("Threat rate:" + str(threatSum / 159570))
print("Insult rate:" + str(insSum / 159570))
print("Identity Hate rate:" + str(idhateSum/ 159570))



y = [toxicSum,sevToxicSum,obsceneSum,threatSum,insSum,idhateSum]
labels = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
plt.suptitle('Distribution of Categories')
plt.figure(figsize = (10,5))
sns.barplot(labels,y)

def evaluate(clf, Xtrain,Xtest, Ytrain,Ytest,comment):
    name = clf.__class__.__name__.split('.')[-1]
    resultList=[]
    for label in labels:
        clf.fit(Xtrain, Ytrain[label])
        y_prediction = clf.predict(Xtest)
        recallScore = recall_score(Ytest[label], y_prediction, average= 'weighted')
        f1 = f1_score(Ytest[label], y_prediction, average= 'weighted')
        result = (name, label, recallScore,f1)
        resultList.append(result)
    
    results = pd.DataFrame(resultList, columns=['name', 'category', 'recallScore', 'f1'])
    display(results)

def crossVal(clf, X,y):
    name = clf.__class__.__name__.split('.')[-1]
    resultList=[]
    for label in labels:
        score = cross_val_score(clf, X, y[label], cv=5, scoring='recall')
        f1 = cross_val_score(clf, X, y[label], cv=5, scoring='f1')
        result = (name, label, score.mean(), f1.mean())
        resultList.append(result)
    
    results = pd.DataFrame(resultList, columns=['name', 'category', 'recallScore', 'f1'])
    results = results.style.set_caption('Cross Validation')    
    display(results)    
    
        

dataset = pd.DataFrame()
toxicWords = []
nonToxicWords = []
hateWords =[]


with open("train.csv", 'r',  encoding="utf8") as file:
    next(file)
    csvreader= csv.reader(file, delimiter=',')
    rowcounter = 0
    for row in csvreader:
        rowcounter+=1
        if rowcounter > 30000:
            break
       
        
        comment = row[1]
        commLength = len(comment)
        comment = re.sub('\s+', ' ', comment)  
        comment = re.sub(r'[^\x00-\x7F]',' ', comment)
        comment = re.sub(r'\d', '',comment) 
        comment = re.sub(r"\n", " ", comment)
        
        punctuation = re.sub("[A-Za-z ]", "", comment)
        punctuationRate = len(punctuation) / commLength
        
        capitals = ''.join([c for c in comment if c.isupper()])
        capitalRate = len(capitals)/ commLength
      
        
        comment = re.sub(r'[^\w\s]','',comment)
        
        
        
        comment = comment.lower()

        toxics = int(row[2])
        severe_toxic = int(row[3])
        obscene = int(row[4])
        threat = int(row[5])
        insult = int(row[6])
        identity_hate = int(row[7])
        

        
        score = (toxics,severe_toxic,obscene,threat,insult,identity_hate)
        wordList = []    
        words = word_tokenize(comment)
        
        for word in words:
            if word not in stop_words and word != '':
                word = lemma.lemmatize(word)
                wordList.append(word)
       
        tags = nltk.pos_tag(words)
        
        tagDict = {}
        
        nouns = []
        verbs = []
        funWord = []
        adverbs = []
        adjectives = []
        
        tagNames = ["nouns, verbs, funWord, adverbs, adjectives"]
        
        for word in tags:
            if(word[1][0] == "N"):
                nouns.append(word[0])
            elif(word[1][0] == "V"):
                verbs.append(word[0])
            elif(word[1] == "DT" or word[1] == "WDT" or word[1] == "PDT" or word[1] == "CC" or word[1] == "WP" or word[1] == "TO" or word[1] == "EX" or word[1] == "IN"):
                funWord.append(word[0])
            elif (word[1][0] == "R" or word[1] == "WRB"):
                adverbs.append(word[0])
            elif (word[1][0] == "J"):
                adjectives.append(word[0])
       
        nounsFreq=0
        verbsFreq=0
        funWordFreq=0
        adverbsFreq=0
        adjectivesFreq=0
        
        if (len(wordList) > 0):
            nounsFreq = len(nouns)/len(wordList)
            verbsFreq = len(verbs)/len(wordList)
            funWordFreq = len(funWord)/len(wordList)
            adverbsFreq = len(adverbs)/len(wordList)
            adjectivesFreq = len(adjectives)/len(wordList)
        summation = 0
        for cat in score:
            summation+=cat
        if (summation == 0):
            nonToxicWords.extend(wordList)
        if(toxics == 1):
            toxicWords.extend(wordList)
        if(identity_hate == 1):
            hateWords.extend(wordList)
        
        
        #print(nounsFreq)
        scoredComment = (' '.join(wordList), nounsFreq , verbsFreq, funWordFreq, adverbsFreq, adjectivesFreq, capitalRate,  punctuationRate)
        
        scoredComments.append(scoredComment)
        scores.append(np.array(score))

fdist = FreqDist(toxicWords)
print(fdist.most_common(50))

fdist = FreqDist(hateWords)
print(fdist.most_common(50))
fdist = FreqDist( nonToxicWords)
print(fdist.most_common(50))





        
dataset = pd.DataFrame(scoredComments, columns=['comments', 'noun freq', 'verb freq','func word freq', 'adverb freq', 'adjective freq', 'capital freq', 'punctuation freq'])

scoreFrame = pd.DataFrame(scores, columns=["toxic","severe_toxic","obscene","threat","insult","identity_hate"])



dataset= pd.concat([dataset,scoreFrame], ignore_index=True, axis =1)
dataset.columns = ['comments', 'noun freq', 'verb freq','func word freq', 'adverb freq', 'adjective freq', 'capital freq', 'punctuation freq', "toxic","severe_toxic","obscene","threat","insult","identity_hate"]
#display(dataset)



col_list = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

nonToxicLen = len(dataset[dataset[col_list].sum(axis=1) == 0])


nonToxicSum = dataset.loc[dataset[col_list].sum(axis=1) == 0, 'capital freq'].sum()
nonToxicCapitalRate = nonToxicSum/nonToxicLen
capitalRateList = [nonToxicCapitalRate]

nonToxicpuncSum= dataset.loc[dataset[col_list].sum(axis=1) == 0, 'punctuation freq'].sum()
nonToxicPunRate = nonToxicpuncSum/nonToxicLen
puncRateList = [nonToxicPunRate]

nounSum= dataset.loc[dataset[col_list].sum(axis=1) == 0, 'noun freq'].sum()
nounRate = nounSum/nonToxicLen
nounRateList = [nounRate]

verbSum= dataset.loc[dataset[col_list].sum(axis=1) == 0, 'verb freq'].sum()
verbRate = verbSum/nonToxicLen
verbRateList = [verbRate]

funcSum= dataset.loc[dataset[col_list].sum(axis=1) == 0, 'func word freq'].sum()
funcRate = funcSum/nonToxicLen
funcRateList = [funcRate]

adverbSum= dataset.loc[dataset[col_list].sum(axis=1) == 0, 'adverb freq'].sum()
adverbRate = adverbSum/nonToxicLen
adverbRateList = [adverbRate]

adjectiveSum= dataset.loc[dataset[col_list].sum(axis=1) == 0, 'adjective freq'].sum()
adjectiveRate = adjectiveSum/nonToxicLen
adjectiveRateList = [adjectiveRate]


for label in labels:
   
    sumCapitals = dataset.loc[dataset[label] == 1, 'capital freq'].sum()
    length = len(dataset[dataset[label] == 1])
    capitalsRate = sumCapitals/length
    capitalRateList.append(capitalsRate)
 
    puncSum= dataset.loc[dataset[label] == 1, 'punctuation freq'].sum()
    puncRate = puncSum/length
    puncRateList.append(puncRate)

    nounSum= dataset.loc[dataset[label] == 1, 'noun freq'].sum()
   
    nounRate = nounSum/length
    nounRateList.append(nounRate)

    
    verbSum= dataset.loc[dataset[label]== 1, 'verb freq'].sum()
    #print(verbSum)
    verbRate = verbSum/length
    #print(verbRate)
    
    verbRateList.append(verbRate)

    funcSum= dataset.loc[dataset[label]== 1, 'func word freq'].sum()
    funcRate = funcSum/length
    funcRateList.append(funcRate)

    adverbSum= dataset.loc[dataset[label] == 1, 'adverb freq'].sum()
    adverbRate = adverbSum/length
    adverbRateList.append(adverbRate)

    adjectiveSum= dataset.loc[dataset[label] == 1, 'adjective freq'].sum()
    adjectiveRate = adjectiveSum/length
    adjectiveRateList.append(adjectiveRate)
    
    
    
    

y = capitalRateList

labels = ["non toxic", "toxic","severe_toxic","obscene","threat","insult","identity_hate"]
plt.figure(figsize = (10,5))
plt.suptitle('Capitals Rate')
sns.barplot(labels,y)

y = puncRateList
plt.figure(figsize=(10,5))
plt.suptitle('Punctuation Rate')
sns.barplot(labels,y)
plt.show()

y = nounRateList
plt.figure(figsize=(10,5))
plt.suptitle('Noun Rate')
sns.barplot(labels,y)
plt.show()

y = verbRateList
plt.figure(figsize=(10,5))
plt.suptitle('Verb Rate')
plot = sns.barplot(labels,y)
plt.show()


y = funcRateList
plt.figure(figsize=(10,5))
plt.suptitle('Function Word Rate')
sns.barplot(labels,y)
plt.show()

y = adverbRateList
plt.figure(figsize=(10,5))
plt.suptitle('Adverb Rate')
sns.barplot(labels,y)
plt.show()

y = adjectiveRateList
plt.figure(figsize=(10,5))
plt.suptitle('Adjective Rate')
sns.barplot(labels,y)
plt.show()

####OPTIONAL CELL

In [ ]:
####TFIDF VECTORIZATION 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from string import punctuation
import csv
import re
import nltk
from sklearn.svm import LinearSVC
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn import tree
from sklearn import metrics
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score,f1_score

# "id","comment_text","toxic","severe_toxic","obscene","threat","insult","identity_hate"

scoredComments = []
scores= []

tweets = []
scoredCommentsTokenized = []
stop_words = set(stopwords.words('english'))

lemma = WordNetLemmatizer()

dataset = pd.read_csv("train.csv")


display(dataset)

print(dataset.describe())


toxicSum = dataset["toxic"].sum()
sevToxicSum = dataset["severe_toxic"].sum()
obsceneSum = dataset["obscene"].sum()
threatSum = dataset["threat"].sum()
insSum = dataset["insult"].sum()
idhateSum = dataset["identity_hate"].sum()

total = 159570
print("Toxic rate:" + str(toxicSum / 159570))
print("Severe Toxic rate:" + str(sevToxicSum / 159570))
print("Obscene rate:" + str(obsceneSum/ 159570))
print("Threat rate:" + str(threatSum / 159570))
print("Insult rate:" + str(insSum / 159570))
print("Identity Hate rate:" + str(idhateSum/ 159570))



y = [toxicSum,sevToxicSum,obsceneSum,threatSum,insSum,idhateSum]
labels = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
plt.figure(figsize = (10,5))
sns.barplot(labels,y)

def evaluate(clf, Xtrain,Xtest, Ytrain,Ytest):
    
    name = clf.__class__.__name__.split('.')[-1]
    resultList=[]
    for label in labels:
        clf.fit(Xtrain, Ytrain[label])
        y_prediction = clf.predict(Xtest)
        recallScore = recall_score(Ytest[label], y_prediction, average= 'weighted')
        f1 = f1_score(Ytest[label], y_prediction, average= 'weighted')
        result = (name, label, recallScore,f1)
        resultList.append(result)
    
    
    results = pd.DataFrame(resultList, columns=['name', 'category', 'recallScore', 'f1'])
    MeanRecall = results['recallScore'].sum()/6
    MeanF1 = results['f1'].sum()/6
    #results = pd.concat([results, (name, "Mean Recall and f1", MeanRecall, MeanF1)])
    
    l = {'name':name, 'category':"Mean Recall and f1", 'recallScore':MeanRecall, 'f1':MeanF1}
   
    results = results.append(l, ignore_index = True,)
    
    display(results)

def crossVal(clf, X,y):
    name = clf.__class__.__name__.split('.')[-1]
    resultList=[]
    for label in labels:
        score = cross_val_score(clf, X, y[label], cv=5, scoring='recall')
        f1 = cross_val_score(clf, X, y[label], cv=5, scoring='f1')
        result = (name, label, score.mean(), f1.mean())
        resultList.append(result)
    
    results = pd.DataFrame(resultList, columns=['name', 'category', 'recallScore', 'f1'])
    
    
    MeanRecall = results['recallScore'].sum()/6
    MeanF1 = results['f1'].sum()/6
    
    l = {'name':name, 'category':"Mean Recall and f1", 'recallScore':MeanRecall, 'f1':MeanF1}
    results = results.append(l, ignore_index = True,)
    
    results = results.style.set_caption('Cross Validation')    
    display(results)
    
    
        
    
#dataset = pd.DataFrame()

with open("train.csv", 'r',  encoding="utf8") as file:
    next(file)
    #skip first entry (name of categories)
    csvreader= csv.reader(file, delimiter=',')
    rowcounter = 0
    #iterate through csv
    for row in csvreader:
        rowcounter+=1
        if rowcounter > 30000:
            break        
        #we are only interested in the first 30000 entries
    
        comment = row[1].lower()
        comment = re.sub('\s+', ' ', comment)  
        comment = re.sub(r'[^\x00-\x7F]','', comment)
        comment = re.sub(r'\d', '',comment) 
        comment = re.sub(r"\n", " ", comment)
        comment = re.sub(r'[^\w\s]','',comment)
        #cleaning document (converted to lowercase)
        #removing punctuation, replacing multiple spaces with one space
        #removing non-ascii chars, newline escape chars, digits
    
        
        toxics = int(row[2])
        severe_toxic = int(row[3])
        obscene = int(row[4])
        threat = int(row[5])
        insult = int(row[6])
        identity_hate = int(row[7])
        #getting scores from csv
        
        score = (toxics,severe_toxic,obscene,threat,insult,identity_hate)
        #tuple containing scores
        
        wordList = []    
        words = word_tokenize(comment)
        #tokenize comment into words
        
        for word in words:
            if word not in stop_words:
                word = lemma.lemmatize(word)
                wordList.append(word)
        #removing stop words and lemmatizing words.
        
        scoredComments.append(' '.join(wordList))
        scores.append(np.array(score))
        
        #rejoin list of words (for use in the tfidf vectorizer) and add it to cleaned comments list
        #add comment's score (categories) to list of scores 

        
#create dataframe for comments and scores
dataset = pd.DataFrame(scoredComments, columns=['comments'])
scoreFrame = pd.DataFrame(scores, columns=["toxic","severe_toxic","obscene","threat","insult","identity_hate"])

#concatenate dataframes
dataset= pd.concat([dataset,scoreFrame], ignore_index=True, axis =1)
dataset.columns = ['comments',"toxic","severe_toxic","obscene","threat","insult","identity_hate"]

display(dataset)

tfIdfvec = TfidfVectorizer(use_idf=True, max_features = 10000, min_df = 10, max_df = 0.60,  ngram_range = (1,1))
X, y = train_test_split(dataset, test_size=0.2,shuffle=True)


print(len(X))
print(len(y))

X_train_vectors_tfidf = tfIdfvec.fit_transform(X["comments"])
X_test_vectors_tfidf = tfIdfvec.transform(y["comments"])
X_eval_vectors_tfidf = tfIdfvec.fit_transform(dataset["comments"])

clf = LinearSVC()
evaluate(clf, X_train_vectors_tfidf, X_test_vectors_tfidf, X, y)
crossVal(clf,X_eval_vectors_tfidf, dataset)

clf = DecisionTreeClassifier()
evaluate(clf, X_train_vectors_tfidf, X_test_vectors_tfidf, X, y)
crossVal(clf,X_eval_vectors_tfidf, dataset)

clf =LogisticRegression()
evaluate(clf, X_train_vectors_tfidf, X_test_vectors_tfidf, X, y)
crossVal(clf,X_eval_vectors_tfidf, dataset)



In [ ]:
####Word2Vec Vectorization

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import scale
from time import time
from gensim.models import Word2Vec
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
import multiprocessing
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from string import punctuation
import csv
import re
import nltk
from sklearn.svm import LinearSVC
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score,f1_score

# "id","comment_text","toxic","severe_toxic","obscene","threat","insult","identity_hate"

# read dataset from file
scoredComments = []
scores= []

tweets = []
scoredCommentsTokenized = []
stop_words = set(stopwords.words('english'))

lemma = WordNetLemmatizer()

dataset = pd.read_csv("train.csv")

labels = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
def evaluate(clf, Xtrain,Xtest, Ytrain,Ytest):
    name = clf.__class__.__name__.split('.')[-1]
    resultList=[]
    for label in labels:
        clf.fit(Xtrain, Ytrain[label])
        
        y_prediction = clf.predict(Xtest)

        recallScore = recall_score(Ytest[label], y_prediction, average= 'weighted')
        f1 = f1_score(Ytest[label], y_prediction, average= 'weighted')
        result = (name, label, recallScore,f1)
        resultList.append(result)
    
    results = pd.DataFrame(resultList, columns=['name', 'category', 'recallScore', 'f1'])
    MeanRecall = results['recallScore'].sum()/6
    MeanF1 = results['f1'].sum()/6
    
    l = {'name':name, 'category':"Mean Recall and f1", 'recallScore':MeanRecall, 'f1':MeanF1}
    results = results.append(l, ignore_index = True,)
    
    display(results)

def crossVal(clf, X,y):
    name = clf.__class__.__name__.split('.')[-1]
    resultList=[]
    for label in labels:
        score = cross_val_score(clf, X, y[label], cv=5, scoring='recall')
        f1 = cross_val_score(clf, X, y[label], cv=5, scoring='f1')
        result = (name, label, score.mean(), f1.mean())
        resultList.append(result)
    
    results = pd.DataFrame(resultList, columns=['name', 'category', 'recallScore', 'f1'])
    
    MeanRecall = results['recallScore'].sum()/6
    MeanF1 = results['f1'].sum()/6
    
    l = {'name':name, 'category':"Mean Recall and f1", 'recallScore':MeanRecall, 'f1':MeanF1}
    results = results.append(l, ignore_index = True,)
    
    results = results.style.set_caption('Cross Validation')    
    display(results)
    
    
tfidfList = []
dataset = pd.DataFrame()

with open("train.csv", 'r',  encoding="utf8") as file:
    next(file)
    csvreader= csv.reader(file, delimiter=',')
    rowcounter = 0
    #for i in range(40000): next(csvreader)
    for row in csvreader:
        rowcounter+=1
        if rowcounter > 30000:
            break
            
        comment = row[1].lower()
        comment = re.sub(r'[^\x00-\x7F]',' ', comment)
        comment = re.sub(r'\d', ' ',comment) 
        comment = re.sub(r"\n", " ", comment)
        comment = re.sub(r'[^\w\s]',' ',comment)
        comment = re.sub('\s+', ' ', comment)  
        
        
        toxics = int(row[2])
        severe_toxic = int(row[3])
        obscene = int(row[4])
        threat = int(row[5])
        insult = int(row[6])
        identity_hate = int(row[7])
        
        score = (toxics,severe_toxic,obscene,threat,insult,identity_hate)
        wordList = []    
        words = word_tokenize(comment)
        
        for word in words:
            if word not in stop_words:
                word = lemma.lemmatize(word)
                wordList.append(word)
        
        scoredComments.append(wordList)
        scores.append(np.array(score))

cores = multiprocessing.cpu_count()

#placing the scores into a dataframe
scoreFrame = pd.DataFrame(scores, columns=["toxic","severe_toxic","obscene","threat","insult","identity_hate"])

#initilizaing the Word2Vec model
w2v_model = Word2Vec(scoredComments,
                     workers = cores-1,
                     min_count=20,
                     window=2,
                     sample=6e-5, 
                     alpha=0.03, 
                     vector_size = 300,
                     min_alpha=0.0007, 
                     negative=20)

def word_vector(tokens, size):
    featureVec = np.zeros((size,), dtype="float32")
    count = 0
    word_index = set(w2v_model.wv.index_to_key)
    
    for word in tokens:
        if word in word_index:
            count+=1
            score = w2v_model.wv.get_vector(word) 
            featureVec = np.add(featureVec, score)
    if count > 0:      
        featureVec = np.divide(featureVec, count)
    return(featureVec)
    

vectorList = []
#get vectors for each comment
for entry in scoredComments:
    vector = word_vector(entry, 300)
    vectorList.append(vector)
    
#store vectors in dataframe
vectorFrame = pd.DataFrame(vectorList)
display(vectorFrame)

X_train, X_test, y_train, y_test = train_test_split(vectorFrame, scoreFrame, test_size=0.2,shuffle=True)

clf = LinearSVC(max_iter = 4000)
evaluate(clf, X_train, X_test, y_train, y_test)
crossVal(clf,vectorFrame, scoreFrame)

clf = DecisionTreeClassifier()
evaluate(clf, X_train, X_test, y_train, y_test)
crossVal(clf,vectorFrame, scoreFrame)

clf = LogisticRegression()
evaluate(clf, X_train, X_test, y_train, y_test)
crossVal(clf,vectorFrame, scoreFrame)


